# i.MX 95 AHAB with U-BOOT

This notebook describes how to build a bootable AHAB image with ELE firmware and U-BOOT bootloader, and how to use NXPELE app.

## 1. Prerequisites

- SPSDK is needed with examples extension. `pip install spsdk[examples]` (Please refer to the installation documentation.)
- This demo was tested with i.MX 95 EVK board with LPDDR5 memory and A0 chip revision.

## 1.1 Images preparation

- to create resulting binary containing AHAB containers, we need to prepare the binaries
- in this section we reproduce the process which is done by the `imx-mkimage` tool
- Obtain all the necessary binaries and put them into inputs directory

## 1.2 U-Boot

In order to use the nxpele app, U-Boot must be built with AHAB support. CONFIG_AHAB_BOOT=y
If you want to use the nxpele over fastboot, also multiplexing of console output to fastboot must be enabled by setting CONFIG_CONSOLE_MUX=y.

## 1.3 Requirements

Download these files from the `https://www.nxp.com/lgfiles/NMG/MAD/YOCTO/<package name>` and put them into the inputs directory.

Primary image container set:
- ELE firmware
- lpddr5 or lpddr4 firmware files with OEI firmware
- CM33 OEI TCM
- CM33 System manager image
- U-Boot SPL
- [Optional] M7 application image

Secondary image container set
- bl31.bin binary (ARM Trusted Firmware)
- U-Boot (built with AHAB support)
- TEE binary

In [12]:
from spsdk.utils.jupyter_utils import YamlDiffWidget

# This env variable sets colored logger output to STDOUT
# Execute this cell to enable execution of the ! line magic
%env JUPYTER_SPSDK=1
%alias execute echo %l && %l
%alias_magic ! execute

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


## 2. AHAB Image


### 2.1 AHAB Template

We can generate the template using the nxpimage ahab get-template command. The command will generate a template. The template is a YAML file that contains the AHAB header and the AHAB container. The AHAB header contains the information about the image, such as the version, the number of containers, and the signature. The AHAB container contains the information about the image, such as the image type, the image version, the image size, and the image signature.

The following command generates the template:

```bash
nxpimage ahab get-template -f mimx9596 -o ahab_template.yaml
```

### 2.2 Exporting of the AHAB image

The AHAB image can be exported using the nxpimage ahab export command. The command will create the AHAB image from the template. The following command creates the AHAB image:

```bash
nxpimage ahab export -c ahab_template.yaml 
```


### 2.4 Primary image container set

Primary image container set consists of the following images:

- ELE firmware
- DDR firmware with OEI
- System manager
- U-Boot SPL
- [Optional] M7 application image

In [7]:
YamlDiffWidget("inputs/mx95_ahab_primary.diffc").html

nxpimage ahab get-template -r a0 -f mimx9596 -o workspace/ahab_template.yaml --force
Creating workspace/ahab_template.yaml template file.


### 2.5 Secondary image container set

Secondary image container set consists of the following images:

- bl31.bin binary (ARM Trusted Firmware)
- U-Boot (built with AHAB support)
- TEE binary


In [8]:
YamlDiffWidget("inputs/mx95_ahab_secondary.diffc").html

nxpimage ahab get-template -r a0 -f mimx9596 -o workspace/ahab_template.yaml --force
Creating workspace/ahab_template.yaml template file.


### 2.6 Exporting of full AHAB image (container set)

U-Boot image consist from two image container sets. The first one contains ELE firmware, DDR firmware with OEI, system manager and U-Boot SPL. This image is loaded using the SDPS protocol. Once the U-Boot SPL is loaded to OCRAM the fastboot is opened and the second container is loaded using the fastboot protocol.

If the image is intended for loading using the UUU, the memory type should be set to serial_downloader. However the type of each individual AHAB should be set to *standard*, because it will be stored in flash memory.

```bash
nxpimage -v bootable-image merge -c u-boot-flash_template.yaml -o flash.bin
```



In [11]:
YamlDiffWidget("inputs/mx95_ahab_bimg.diffc").html

nxpimage bootable-image get-templates -f mimx9596 -o workspace --force
Creating workspace/bootimg_mimx9596_serial_downloader.yaml template file.
Creating workspace/bootimg_mimx9596_flexspi_nor.yaml template file.
Creating workspace/bootimg_mimx9596_emmc.yaml template file.
Creating workspace/bootimg_mimx9596_sd.yaml template file.
Creating workspace/bootimg_mimx9596_recovery_spi.yaml template file.


In [2]:
U_BOOT_PRIMARY = "inputs/mx95_uboot_primary.yaml"
U_BOOT_SECONDARY = "inputs/mx95_uboot_secondary.yaml"
U_BOOT_FLASH_BOOT_CFG = "inputs/bootimg_mx95_serial_downloader.yaml"
U_BOOT_FLASH_BOOT = "outputs/flash.bin"

VERBOSITY = "-v"

# EXPORT U-BOOT IMAGE
%! nxpimage -v bootable-image merge --config $U_BOOT_FLASH_BOOT_CFG --output $U_BOOT_FLASH_BOOT

nxpimage -v bootable-image merge --config bootimg_mx95_serial_downloader.yaml --output outputs/flash.bin 
INFO:spsdk.apps.nxpimage:Created Bootable Image:
Name:      Bootable Image for mimx9596
Starts:    0x0
Ends:      0x26bbff
Size:      Size: 2.4 MiB; 2,538,496 B
Alignment: 1 B
Pattern:zeros
Memory type: MemoryType.SERIAL_DOWNLOADER
Revision: latest

INFO:spsdk.apps.nxpimage:Created Bootable Image memory map:

+==0x0000_0000= Bootable Image for mimx9596 =====================+
|                   Size: 2.4 MiB; 2,538,496 B                   |
|   Memory type: MemoryType.SERIAL_DOWNLOADER Revision: latest   |
|                         Pattern: zeros                         |
|+==0x0000_0000= primary_image_container_set ===================+|
||                  Size: 852.0 kiB; 872,448 B                  ||
||                  AHAB Image for mimx9596_a0                  ||
||                        Pattern: zeros                        ||
||+==0x0000_0000= AHAB Containers =============

In [8]:
# Set the boot mode to Cortex-M Serial Downloader 1001 and download the files using the UUU tool

%! nxpuuu $VERBOSITY write -f mimx9596 -b emmc $U_BOOT_FLASH_BOOT

nxpuuu -v write -f mimx9596 -b emmc outputs/flash.bin 
SDPS: boot -f outputs/flash.bin


SDPV: write -f outputs/flash.bin -skipspl


SDPV: jump


Done


Now change the boot mode to Cortex-M EMMC 1010 and reset the board. Find the serial port that belongs to U-Boot console and interrupt the boot. 
When the console is switched to U-Boot menu, we can use the *nxpele* tool to communicate with the ELE.


In [9]:
%! nxpele -f mimx9596 -p COM180 -d uboot_serial get-info

nxpele -f mimx9596 -p COM180 -d uboot_serial get-info 
ELE get info ends successfully:
Command:          0xda
Version:          2
Length:           160
SoC ID:           MX95 - 0x9500
SoC version:      A100
Life Cycle:       OEM_OPEN - 0x0010
SSSM state:       4
UUID:             e1e6e6f98b514065bd991d922e30c659
SHA256 ROM PATCH: e32d8de33ee1ac953d78379b9d6c1523544e3b77a9848d1d1af9374f678a81c1
SHA256 FW:        eb3f39a5dccafb3335e8e3e340d3e5f06b386666036a30affb660a301b5ed420
Advanced information:
  OEM SRKH:       0000000000000000000000000000000000000000000000000000000000000000
  IMEM state:     This is non-existing tag(0x0) from enum: EleImemState - 0x00
  CSAL state:     EdgeLock secure enclave random context initialization succeed - 0x02
  TRNG state:     TRNG entropy is valid and ready to be read - 0x03
  OEM PQC SRKH:  00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000



In [11]:
%! nxpele -f mimx9596 -p COM180 -d uboot_serial generate-keyblob DEK -a AES_CBC --key-id 0 --key 00000000000000000000000000000000 --key-size 128

nxpele -f mimx9596 -p COM180 -d uboot_serial generate-keyblob DEK -a AES_CBC --key-id 0 --key 00000000000000000000000000000000 --key-size 128 
ELE generate DEK key blob ends successfully:
00480081011003002021528bf3419a5498d94d8f73333fb48e1925049106cc0a28d5fb19a2c48a5797b509156279678db9b3a304af8d073ed2f8937a24e80a07381f79cede1d0577
